In [1]:
# Script for importing netcdf, finding optimal shuffle settings for encoding, and then saving the dataset with these settings
import numpy as np
import xarray as xr
import os   
import copy
import re
import datetime

In [ ]:
map_file_in = r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR1\storm1_lidar1_polar_10sInterval - Copy.nc'
map_file_out1 = r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR1\r no shuffle.nc'
map_file_out2 = r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR1\r shuffle.nc'
map_file_out3 = r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR1\i no shuffle.nc'
map_file_out4 = r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR1\i shuffle.nc'
map_file_out5 = r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR1\r_andi_i shuffle fillvalue_0.nc'

ds = xr.open_dataset(map_file_in)
# ds_r = ds.copy()
# ds_i = ds.copy()

# ds_r = ds_r[['radius_lidar']]  # keep only the 'r' variable
# ds_i = ds_i[['intensity']]  # keep only the 'r' variable

# # Radius
# compression = {var: {"zlib": True, "complevel": 5} for var in list(ds_r.data_vars) + list(ds_r.coords)}  # temporary dict, with only compression settings

# encoding = {'radius_lidar': { 'dtype': 'uint16', 'shuffle': False} }
# for var, comp in compression.items():  # for each variable in the dataset, 
#     if var in encoding:                # if the variable already has an encoding, update it with the compression settings
#         encoding[var].update(comp)
#     else:                              # if the variable does not have an encoding yet, add it 
#         encoding[var] = comp
# ds_r.to_netcdf(map_file_out1, encoding=encoding)  # Save the dataset with deflate compression

# encoding = {'radius_lidar': { 'dtype': 'uint16', 'shuffle': True} }
# for var, comp in compression.items():  # for each variable in the dataset, 
#     if var in encoding:                # if the variable already has an encoding, update it with the compression settings
#         encoding[var].update(comp)
#     else:                              # if the variable does not have an encoding yet, add it 
#         encoding[var] = comp
# ds_r.to_netcdf(map_file_out2, encoding=encoding)  # Save the dataset with deflate compression


# # Intensity ------------------------------
# compression = {var: {"zlib": True, "complevel": 5} for var in list(ds_i.data_vars) + list(ds_i.coords)}  # temporary dict, with only compression settings

# encoding = {'intensity': { 'dtype': 'uint8', 'shuffle': False} }
# for var, comp in compression.items():  # for each variable in the dataset, 
#     if var in encoding:                # if the variable already has an encoding, update it with the compression settings
#         encoding[var].update(comp)
#     else:                              # if the variable does not have an encoding yet, add it 
#         encoding[var] = comp
# ds_i.to_netcdf(map_file_out3, encoding=encoding)  # Save the dataset with deflate compression

# encoding = {'intensity': { 'dtype': 'uint8', 'shuffle': True} }
# for var, comp in compression.items():  # for each variable in the dataset, 
#     if var in encoding:                # if the variable already has an encoding, update it with the compression settings
#         encoding[var].update(comp)
#     else:                              # if the variable does not have an encoding yet, add it 
#         encoding[var] = comp
# ds_i.to_netcdf(map_file_out4, encoding=encoding)  # Save the dataset with deflate compression

# Conclusion: shuffle better
# variable time_string from ds
# ds = ds.drop_vars(['time_string'])  # drop the time_string variable, as it is not needed for the compression test

compression = {var: {"zlib": True, "complevel": 5} for var in list(ds.data_vars) + list(ds.coords)}  # temporary dict, with only compression settings

encoding = {'radius_lidar': { 'dtype': 'uint16', '_FillValue': 0, 'shuffle': True},
            'intensity':    { 'dtype': 'uint8',  '_FillValue': 0, 'shuffle': True} }
for var, comp in compression.items():  # for each variable in the dataset, 
    if var in encoding:                # if the variable already has an encoding, update it with the compression settings
        encoding[var].update(comp)
    else:                              # if the variable does not have an encoding yet, add it 
        encoding[var] = comp
ds.to_netcdf(map_file_out5, encoding=encoding)  # Save the dataset with deflate compression

In [ ]:
map_file_in = r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR1\storm1_lidar1_polar_10sInterval - Copy.nc'
map_file_out1 = r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR1\r shuffle chunk_15240_240_1_1.nc'

ds = xr.open_dataset(map_file_in)
ds_r = ds.copy()
ds_r = ds_r[['radius_lidar']]  # keep only the 'r' variable

# Radius
compression = {var: {"zlib": True, "complevel": 4} for var in list(ds_r.data_vars) + list(ds_r.coords)}  # temporary dict, with only compression settings

encoding = {'radius_lidar': { 'dtype': 'uint16',  '_FillValue': 0, 'chunksizes': (1, 1, 240, 15240), 'shuffle': True} }
for var, comp in compression.items():  # for each variable in the dataset, 
    if var in encoding:                # if the variable already has an encoding, update it with the compression settings
        encoding[var].update(comp)
    else:                              # if the variable does not have an encoding yet, add it 
        encoding[var] = comp
ds_r.to_netcdf(map_file_out1, encoding=encoding)  # Save the dataset with deflate compression

# Conclusion chunks: focus on time. 1 or 4 profiles does not matter. 240 or 720 angles does not matter. 240 angles better than 1. So 
# minimum total chunk size? And 5080 times is slightly better than 15240, so you can have too much time together.

In [3]:
#map_file_in = r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR1\storm1_lidar1_polar_10sInterval - Copy.nc'
map_file_in = r'C:\Matlab\hybrid_dunes\storm1_lidar1_polar - Copy.nc'
map_file_out1 = r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR1\full_storm shuffle chunk_28800_180.nc'
map_file_out2 = r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR1\full_storm shuffle chunk_28800_7200.nc'
map_file_out3 = r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR1\r full_storm shuffle chunk_28800_120.nc'
map_file_out4 = r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR1\r full_storm shuffle chunk_28800_90_1_3.nc'
map_file_out5 = r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR1\r full_storm shuffle chunk_28800_30_1_3.nc'

#ds = xr.open_dataset(map_file_in, chunks = "{'time': 15240, 'profile_number': 1, 'obs_number': 90, 'echos': 1}")  # load the dataset with specified chunking
ds = xr.open_dataset(map_file_in, chunks = "auto")  # load the dataset with specified chunking
#ds = xr.open_dataset(map_file_in)  # load the dataset with specified chunking

#ds = ds[['radius_lidar']]  # keep only the 'r' variable

# Radius
compression = {var: {"zlib": True, "complevel": 4} for var in list(ds.data_vars) + list(ds.coords)}  # temporary dict, with only compression settings

#encoding = {'radius_lidar': { 'dtype': 'uint16',  '_FillValue': 0, 'chunksizes': (1, 1, 240, 15240), 'shuffle': True} }
#encoding = {'radius_lidar': { 'dtype': 'uint16', '_FillValue': 0, 'chunksizes': (1, 1, 180, 15240), 'shuffle': True},
#            'intensity':    { 'dtype': 'uint8',  '_FillValue': 0, 'chunksizes': (1, 1, 180, 15240), 'shuffle': True} }
encoding1 = {'radius_lidar': { 'dtype': 'uint16', '_FillValue': 0, 'chunksizes': (1, 1, 180, 28800), 'shuffle': True},
             'intensity':    { 'dtype': 'uint8',  '_FillValue': 0, 'chunksizes': (1, 1, 180, 28800), 'shuffle': True} }
encoding2 = {'radius_lidar': { 'dtype': 'uint16', '_FillValue': 0, 'chunksizes': (1, 1, 720, 28800), 'shuffle': True},
             'intensity':    { 'dtype': 'uint8',  '_FillValue': 0, 'chunksizes': (1, 1, 720, 28800), 'shuffle': True} }
encoding3 = {'radius_lidar': { 'dtype': 'uint16', '_FillValue': 0, 'chunksizes': (1, 1, 120, 28800), 'shuffle': True},
             'intensity':    { 'dtype': 'uint8',  '_FillValue': 0, 'chunksizes': (1, 1, 120, 28800), 'shuffle': True} }
encoding4 = {'radius_lidar': { 'dtype': 'uint16', '_FillValue': 0, 'chunksizes': (3, 1, 90, 28800), 'shuffle': True},
             'intensity':    { 'dtype': 'uint8',  '_FillValue': 0, 'chunksizes': (3, 1, 90, 28800), 'shuffle': True} }
encoding5 = {'radius_lidar': { 'dtype': 'uint16', '_FillValue': 0, 'chunksizes': (3, 1, 30, 28800), 'shuffle': True},
             'intensity':    { 'dtype': 'uint8',  '_FillValue': 0, 'chunksizes': (3, 1, 30, 28800), 'shuffle': True} }

for var, comp in compression.items():  # for each variable in the dataset, 
    if var in encoding1:                # if the variable already has an encoding, update it with the compression settings
        encoding1[var].update(comp)
    else:                              # if the variable does not have an encoding yet, add it 
        encoding1[var] = comp

    if var in encoding2:                # if the variable already has an encoding, update it with the compression settings
        encoding2[var].update(comp)
    else:                              # if the variable does not have an encoding yet, add it 
        encoding2[var] = comp

    if var in encoding3:                # if the variable already has an encoding, update it with the compression settings
        encoding3[var].update(comp)
    else:                              # if the variable does not have an encoding yet, add it 
        encoding3[var] = comp

    if var in encoding4:                # if the variable already has an encoding, update it with the compression settings
        encoding4[var].update(comp)
    else:                              # if the variable does not have an encoding yet, add it 
        encoding4[var] = comp

    if var in encoding5:                # if the variable already has an encoding, update it with the compression settings
        encoding5[var].update(comp)
    else:                              # if the variable does not have an encoding yet, add it 
        encoding5[var] = comp

print("start saving file 1 at", datetime.datetime.now()) # print  current time
ds.to_netcdf(map_file_out1, encoding=encoding1, compute=True)  
print("Saved dataset 1 at", datetime.datetime.now()) 

ds.to_netcdf(map_file_out2, encoding=encoding2, compute=True)  # Save the dataset with deflate compression
print("Saved dataset 2 at", datetime.datetime.now()) 

# ds.to_netcdf(map_file_out3, encoding=encoding3, compute=True)  # Save the dataset with deflate compression
# print("Saved dataset 3 at", datetime.datetime.now()) 

# ds.to_netcdf(map_file_out4, encoding=encoding4, compute=True)  # Save the dataset with deflate compression
# print("Saved dataset 4 at", datetime.datetime.now())

# ds.to_netcdf(map_file_out5, encoding=encoding5, compute=True)  # Save the dataset with deflate compression
# print("Saved dataset 5 at", datetime.datetime.now())

start saving file 1 at 2025-08-07 13:13:51.568585
Saved dataset 1 at 2025-08-07 13:45:23.844315
Saved dataset 2 at 2025-08-07 15:38:59.245318


In [ ]:
# To change
# drop time_string
# radius =double(radius)*1000, add scale factor
# time_num: chunk
# change metadata radius
# try to add time coordinate, otherwise change metadata time_num
# check first and last time: in open air?
# make beam_angle 720*16, drop ini_beam_angle

# To try
# re-order dimensions

<xarray.Dataset> Size: 4GB
Dimensions:         (time: 15240, profile_number: 16, echos: 3, obs_number: 720)
Dimensions without coordinates: time, profile_number, echos, obs_number
Data variables:
    time_num        (time) datetime64[ns] 122kB dask.array<chunksize=(15240,), meta=np.ndarray>
    time_string     (time) <U23 1MB dask.array<chunksize=(15240,), meta=np.ndarray>
    file_name       (time) <U42 3MB dask.array<chunksize=(15240,), meta=np.ndarray>
    profile_angle   (profile_number) float32 64B dask.array<chunksize=(16,), meta=np.ndarray>
    ini_beam_angle  (profile_number) float32 64B dask.array<chunksize=(16,), meta=np.ndarray>
    radius_lidar    (echos, profile_number, obs_number, time) float32 2GB dask.array<chunksize=(1, 4, 240, 5080), meta=np.ndarray>
    intensity       (echos, profile_number, obs_number, time) float32 2GB dask.array<chunksize=(1, 6, 240, 5080), meta=np.ndarray>
Attributes:
    name:                   storm1_lidar1_polar polar
    summary:            

In [4]:
map_file_in = r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR1\storm1_lidar1_polar - Copy.nc'
map_file_out = r'O:\HybridDune experiment\2024-12-18 to 2024-12-20, Storm 1\Lidars\20241220_LiDAR1\storm1_lidar1_polar - new.nc'

ds = xr.open_dataset(map_file_in, chunks = "auto")  # load the dataset with specified chunking
#print(ds.radius_lidar)

# cast radius_lidar as float64
radius_attrs = ds['radius_lidar'].attrs.copy()
ds['radius_lidar'] = ds['radius_lidar'].astype('float64') / 1000 # convert to meters. Removes attributes, so reset/correct them below
#ds['radius_lidar'].attrs = radius_attrs  # restore attributes
ds['radius_lidar'].attrs = {'long name': 'distance (radius) from lidar to point', 
                            'units': 'm', 
                            'comment': 'radius, part of polar coordinates of points. Polar angles per point can be calculated from the profile_angle and beam_angle', 
                            'dimensions': 'T x 720 x 16 x 3 (Matlab) or reverse order (Python XArray), for number of point clouds x (360°x0.5° angular resolution) x 16 profiles x 3 echos. The lidar registers at most 3 echos at each angle, usually less.'}

# rename coordinate time to t
ds = ds.rename({'time': 't'})  # rename time coordinate to t
ds['t'].attrs['comment'] = 'UTC+1: local wintertime'  # change long_name of t to time

# assign values in time_num to coordinate of t
ds = ds.assign_coords(t=ds.time_num.values)  # assign values in time_num to coordinate of t

# make beam_angle instead of ini_beam_angle
angles = np.arange(0, 360, 0.5)  # create angles
ini_beam_angle = ds['ini_beam_angle'].values  # get ini_beam_angle values. 1x16 matrix
beam_angle = np.repeat(ini_beam_angle[np.newaxis, :], 720, axis=0)  # repeat ini_beam_angle 720 times, for each angle. gives 720x16 matrix
beam_angle = beam_angle + np.repeat(angles[:, np.newaxis], 16, axis=1)  # repeat angles for each profile, gives 720x16 matrix


ds['beam_angle'] = (('profile_number', 'obs_number'), beam_angle.T)  # add beam_angle as a new coordinate
ds['beam_angle'].attrs = {'long name': 'angle of the lidar beam', 
                          'units': 'degrees', 
                          'comment': 'the angular resolution is 0.5°. So every next measurement within a profile is exactly 0.5 degrees later. The angle of the first point differs slightly between profiles. Angles are identical between epochs (i.e. over time).'}

# Drop variables: time_string, time_num, ini_beam_angle
ds = ds.drop_vars('time_string')     # drop time_string variable
ds = ds.drop_vars('time_num')        # drop time_num variable
ds = ds.drop_vars('ini_beam_angle')  # drop ini_beam_angle variable
#ds = ds[['beam_angle']]  # keep only the 'r' variable

# Radius
compression = {var: {"zlib": True, "complevel": 4} for var in list(ds.data_vars) + list(ds.coords)}  # temporary dict, with only compression settings
encoding = {'radius_lidar': { 'scale_factor': 0.001, 'dtype': 'uint16', '_FillValue': 0, 'chunksizes': (1, 1, 120, 28800)},
             'intensity':   {                        'dtype': 'uint8',  '_FillValue': 0, 'chunksizes': (1, 1, 120, 28800)} }  # encoding for the dataset  

for var, comp in compression.items():  # for each variable in the dataset, 
    if var in encoding:                # if the variable already has an encoding, update it with the compression settings
        encoding[var].update(comp)
    else:                              # if the variable does not have an encoding yet, add it 
        encoding[var] = comp

ds.to_netcdf(map_file_out, encoding=encoding, compute=True)  